<a href="https://colab.research.google.com/github/alexanderportilla96/Sy2025II/blob/main/Solucion_parcial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>